In [1]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [3]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "v2")

In [7]:
# Set the tickers for both the bond and stock portion of the portfolio
#SPY = S&P500
#AGG= Core US Aggregate Bond ETF
#QQQ= QQQ tracks a modified-market-cap-weighted index of 100 NASDAQ-listed stocks.
#VTV= Value ETF (Provides a convenient way to match the performance of many of the nation’s largest value stocks), 
#IEMG = Emerging Market ETF
#IWM = Russell 2000 index ETF (small cap)
tickers = ['SPY', 'AGG', 'QQQ','VTV', 'IEMG', 'IWM']

# Set timeframe to 1D 
timeframe = "1Day"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2017-01-01T00:00:00", tz = "America/Los_Angeles").isoformat()
end_date = pd.Timestamp("2021-12-31T00:00:00", tz = "America/Los_Angeles").isoformat()
limit_rows = 10000



In [8]:
#function that accepts a variable string ticker for pulling the dataframe from alpaca
def puller(ticker):
    df1 = alpaca.get_bars(ticker, 
                          timeframe, 
                          start=start_date, 
                          end=end_date, 
                          limit=limit_rows).df
    return df1

#function for structuring dataframe so MCsimulation will accept as input
def structuredata(df, ticker):    
    col_names = [(ticker, x) for x in df.columns]
    df.columns = pd.MultiIndex.from_tuples(col_names)
    return df

#function to merge two dfs into MCsimulation format
def merger(df1, df2):
        df_merged = pd.merge(df1, df2, how = "inner", left_index=True, right_index=True)
        return df_merged

#run function that accepts a list of strings and other defined other variables needed for structuring the combined data
def run(list, currentdf, ticker):
        if(list.index(ticker) == 0):
                df = puller(ticker)
                df = structuredata(df, ticker)
                #display(df.head())
                #print('hi')
                return df
        else:        
                df = puller(ticker)
                df = structuredata(df, ticker)
                df = merger(currentdf, df)
                #display(df.head())
                #print('ho')
                return df

MCdf = pd.DataFrame()

for i in tickers:
        MCdf = run(tickers, MCdf, i)

display(MCdf.head())


SPY                                        \
                             open      high       low   close    volume   
timestamp                                                                 
2017-01-03 05:00:00+00:00  225.07  225.8300  223.8837  225.24  91366522   
2017-01-04 05:00:00+00:00  225.64  226.7500  225.6100  226.58  78744433   
2017-01-05 05:00:00+00:00  226.28  226.5800  225.4800  226.40  78379012   
2017-01-06 05:00:00+00:00  226.53  227.7500  225.9000  227.21  71559922   
2017-01-09 05:00:00+00:00  226.90  227.0701  226.4163  226.46  46939676   

                                                      AGG                    \
                          trade_count        vwap    open    high       low   
timestamp                                                                     
2017-01-03 05:00:00+00:00      314573  224.642686  107.73  108.22  107.7300   
2017-01-04 05:00:00+00:00      259503  226.196593  108.16  108.21  108.0300   
2017-01-05 05:00:00+00:00      218284  226.251697  108.37  108.68  108.2201   
2017-01-06 05:00:00+00:00      235983  227.113087  108.43  108.50  108.2600   
2017-01-09 05:00:00+00:00      158755  226.731966  108.54  108.54  108.4300   

                           ...     IEMG                            IWM  \
                           ...   volume trade_count       vwap    open   
timestamp                  ...                                           
2017-01-03 05:00:00+00:00  ...  7008609       21534  42.880855  136.52   
2017-01-04 05:00:00+00:00  ...  5154644       22121  43.244844  136.05   
2017-01-05 05:00:00+00:00  ...  5440393       16873  43.720961  137.50   
2017-01-06 05:00:00+00:00  ...  7137810       27035  43.561160  136.41   
2017-01-09 05:00:00+00:00  ...  4767580       17904  43.551103  135.54   

                                                                           \
                               high     low   close    volume trade_count   
timestamp                                                                   
2017-01-03 05:00:00+00:00  136.8295  134.50  135.52  29792446      125905   
2017-01-04 05:00:00+00:00  137.9600  136.01  137.78  34499365      139192   
2017-01-05 05:00:00+00:00  137.7600  135.51  136.19  30286876      123714   
2017-01-06 05:00:00+00:00  136.7100  135.68  135.69  23435995       97449   
2017-01-09 05:00:00+00:00  135.7200  134.50  134.80  24013865       99844   

                                       
                                 vwap  
timestamp                              
2017-01-03 05:00:00+00:00  135.469157  
2017-01-04 05:00:00+00:00  137.292887  
2017-01-05 05:00:00+00:00  136.444733  
2017-01-06 05:00:00+00:00  136.110436  
2017-01-09 05:00:00+00:00  135.105597  

[5 rows x 42 columns]

In [ ]:
# Configure the Monte Carlo simulation to forecast 30 years cumulative returns
# The weights should be split 40% to AGG and 60% to SPY.
# Run 500 samples.
MC_equal_weight = MCSimulation(
    portfolio_data = MCdf,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
# Printing the first five rows of the simulation input data
MC_equal_weight.portfolio_data.head()

In [ ]:
MC_equal_weight.calc_cumulative_return()

In [ ]:
MC_equal_weight_plot = MC_6040_weight.plot_simulation()